In [2]:
##########----- Reused code -----##########
# Last update: Sept 13, 2020
# By: Huitian Diao
###########################################

library(stringr)

code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-2)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "function_GSEA.R"))
base_dir <- gsub("/codes_local", "", code_dir)
base_dir

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/0_Acute-Chronic"

## Resampled Louvain cluster GSEA

In [3]:
##########----- Parameters-----##########
analysis_name <- c("0", "resampled")
genome <- "mm"
#########################################
analysis_subdir <- paste(analysis_name[1], "Scanpy_out", analysis_name[2], sep="_")
sum.dir <- file.path(base_dir, '1_Scanpy', analysis_subdir, "3_GSEA")
dir.create(sum.dir, showWarnings = FALSE, recursive=TRUE)
setwd(sum.dir)


###----- Select gene signature
gs.dir <- file.path('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm')
gs.pattern <- paste("_", genome, "_sigs.csv", sep="")
gs.files <- list.files(gs.dir, pattern=gs.pattern, full.names=TRUE)
gs.files <- c('/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/2_IL2RAKO_Nascent/IL2_dependent_mm_sigs.csv') #, gs.files

In [4]:
z.use.keys <- c('louvain_celltype_general', 'louvain', 'cell_type', 
                'louvain_celltype', 'louvain_celltype_general_withNAV') 

#####---------- For each classification method
for (z.use.key in z.use.keys) {
    z.file.name <- paste(z.use.key, 'mean_scaled_expr.csv', sep="_")
    z.file <- file.path(base_dir, '1_SCANPY', analysis_subdir, "1_avg_expr", z.file.name)
    z.tb <- read_csv(z.file)
    
    #####---------- For each signature collection, calculate GSEA & summarize
    for (gs.file in gs.files) {
        # Create sub directory for signature group
        gs.file.base <- gsub(gs.pattern, "",basename(gs.file))

        sep.dir <- paste(z.use.key, "sep", sep="_")
        wk.dir <- file.path(sum.dir, sep.dir, gs.file.base)
        dir.create(wk.dir, showWarnings = FALSE, recursive=TRUE)
        setwd(wk.dir)

        ###----- Run GSEA
        for (i in tail(colnames(z.tb), -1)){
            useGroup <- i
            outName <- i
            gsea <- tryCatch({
                    GSEA_analysis(z.file, useGroup, outName, gs.file, 0.5)
                }, error=function(cond) {
                    message(paste(z.use.key, gs.file, i, sep='; '))
                })
        }

        ###----- Summarize
        setwd(sum.dir)

        gsea.sum.name <- paste(z.use.key, "gsea", gs.file.base, sep="_")

        gs.all.tb <- read_csv(gs.file)
        path.order <- str_sort(unique(gs.all.tb$gs_name), numeric = TRUE)
        gs.file.ext <- basename(gs.file)
        gs.file.ext <- paste("---", gs.file.ext, sep="")

        comp.order <- sort(tail(colnames(z.tb), -1))

        gsea.files <- list.files(path = wk.dir, pattern = "*.csv", full.names = TRUE)
        gsea.file.base <- gsub(gs.file.ext, "", basename(gsea.files))
        comp.order <- intersect(comp.order, gsea.file.base)
        comp.order <- str_sort(comp.order, numeric = TRUE)
        print(comp.order)

        sum_df <- GSEA_sum(gsea.files, gsea.sum.name, 40, 20, TRUE, comp.order, TRUE, path.order,
                           FALSE, None, TRUE)
    }
}


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  gene_name = col_character()
)
ℹ Use `spec()` for the full column specifications.



── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  gene_name = col_character()
)
ℹ Use `spec()` for the full column specifications.


Warning message:
“Unknown columns: `gene_names`”


[1] 78



── Column specification ────────────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  gene_name = col_character()
)
ℹ Use `spec()` for the full column specifications.


Warning message:
“Unknown columns: `gene_names`”


[1] 402



── Column specification ────────────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ────────────────────────────────────────────────────────
cols(
  .default = col_double(),
  gene_name = col_character()
)
ℹ Use `spec()` for the full column specifications.


Warning message:
“Unknown columns: `gene_names`”


...


── Column specification ────────────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)


preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...


── Column specification ────────────────────────────────────────────────────────
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)




 [1] "0_A"  "0_C"  "1_A"  "1_C"  "2_A"  "2_C"  "3_A"  "3_C"  "4_A"  "4_C" 
[11] "5_A"  "5_C"  "6_A"  "6_C"  "7_A"  "7_C"  "8_A"  "8_C"  "9_A"  "9_C" 
[21] "10_A" "10_C"



── Column specification ────────────────────────────────────────────────────────
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_logical(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_logical(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col


── Column specification ────────────────────────────────────────────────────────
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_double(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_double(),
  NES = col_double(),
  pvalue = col_double(),
  p.adjust = col_double(),
  qvalues = col_logical(),
  rank = col_double(),
  leading_edge = col_character(),
  core_enrichment = col_character()
)



── Column specification ────────────────────────────────────────────────────────
cols(
  ID = col_character(),
  Description = col_character(),
  setSize = col_double(),
  enrichmentScore = col_

# A tibble: 6 x 8
  comparison pathway    NES    padj    pval leadingEdge_sig… mlog10padj
  <chr>      <chr>    <dbl>   <dbl>   <dbl>            <dbl>      <dbl>
1 0_A        48h_IL…  1.61  0.0126  0.00632               20      1.90 
2 0_A        48h_IL…  1.40  0.0160  0.0160                24      1.79 
3 0_C        48h_IL…  1.51  0.00224 0.00112               26      2.65 
4 0_C        48h_IL…  1.62  0.00317 0.00317               23      2.50 
5 1_A        48h_IL… -1.02  0.677   0.382                 20      0.169
6 1_A        48h_IL…  0.857 0.677   0.677                 15      0.169
# … with 1 more variable: mlog10pval <dbl>
